In [2]:
import numpy as np
import re, json, csv
import torch
from matplotlib import pyplot as plt
from pymongo import MongoClient
from tqdm.notebook import tqdm
from pprint import PrettyPrinter
from typing import Union, Iterable
from bson.objectid import ObjectId
from torchmetrics.text.bert import BERTScore
from torchmetrics.text.rouge import ROUGEScore
from torchmetrics.text.bleu import BLEUScore
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoConfig, AutoTokenizer
from torchmetrics.functional import pairwise_cosine_similarity
from torchmetrics.functional.text.bert import _get_precision_recall_f1
from functools import partial
from collections import Counter, defaultdict
from itertools import chain
from math import log
from multiprocessing import Pool
from typing import List
from pymongo import MongoClient
from typing import Set
from collections import OrderedDict

pprint = PrettyPrinter().pprint

In [1]:
DEVICE = "cpu"

In [3]:
db = MongoClient("mongodb://localhost:27017")["cuda_snippets"]
train_db = db["train"]
validation_db = db["validation"]

### Get model

### Calculate ROUGE

In [4]:
def compute_rouge_score(target_sentences : Iterable[str], pred_senteces : Iterable[str], tokenizer):
    tokenizer_fn = lambda x: tokenizer.convert_ids_to_tokens(tokenizer.encode(x))
    rouge_metric = ROUGEScore(tokenizer=tokenizer_fn)
    # rouge_metric.update(pred_senteces, target_sentences)
    return rouge_metric(pred_senteces, target_sentences)

### Calculate BLEU

In [5]:
def compute_bleu_score(target_sentences : Iterable[str], pred_senteces : Iterable[str], tokenizer):
    bleu_metric = BLEUScore(tokenizer=tokenizer)
    bleu_metric.update(pred_senteces, [[s] for s in target_sentences])
    return bleu_metric.compute()

### Calculate BERT score

In [6]:
def process(a, tokenizer=None):
    if tokenizer is not None:
        a = tokenizer.encode(a, max_length=512, truncation=True, add_special_tokens=False)
    return set(a)

def get_idf_dict(arr, tokenizer, nthreads=4):
    idf_count = Counter()
    num_docs = len(arr)

    process_partial = partial(process, tokenizer=tokenizer)

    if nthreads > 0:
        with Pool(nthreads) as p:
            idf_count.update(chain.from_iterable(p.map(process_partial, arr)))
    else:
        idf_count.update(chain.from_iterable(map(process_partial, arr)))

    idf_dict = defaultdict(lambda: log((num_docs + 1) / (1)))
    idf_dict.update(
        {idx: log((num_docs + 1) / (c + 1)) for (idx, c) in idf_count.items()}
    )
    return idf_dict

class TextDataset:
    def __init__(self, target_sentences, pred_sentences) -> None:
        self.target_sentences = target_sentences
        self.pred_sentences = pred_sentences
    
    def __len__(self):
        return len(self.target_sentences)
    
    def __getitem__(self, i):
        return self.target_sentences[i], self.pred_sentences[i]
    
class CollateFn:
    def __init__(self, tokenizer) -> None:
        self.tokenizer = tokenizer
    
    def __call__(self, samples : list):
        targets, preds = zip(*samples)
        
        targets_pt = self.tokenizer(targets, max_length=512, truncation=True, return_tensors="pt", add_special_tokens=False, padding=True)
        preds_pt = self.tokenizer(preds, max_length=512, truncation=True, return_tensors="pt", add_special_tokens=False, padding=True)

        return (targets_pt, targets), (preds_pt, preds)

def compute_bert_score(target_sentences : Iterable[str], pred_sentences : Iterable[str], tokenizer, embedd_layer : torch.nn.Embedding, idf_scores : dict = None, batch_size=64):

    if idf_scores is None:
        idf_scores = get_idf_dict(target_sentences, tokenizer)
    precs = []
    recals = []
    f1s = []
    
    dataset = TextDataset(target_sentences, pred_sentences)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=CollateFn(tokenizer))
    
    for i, ((target_pt, target_str), (pred_pt, pred_str)) in enumerate(tqdm(dataloader)):

        pred_embedd = embedd_layer(pred_pt["input_ids"]).view((pred_pt["input_ids"].size(0),1,pred_pt["input_ids"].size(1), -1))
        target_embedd = embedd_layer(target_pt["input_ids"]).view((target_pt["input_ids"].size(0),1,target_pt["input_ids"].size(1), -1))
        
        pred_idfs = [tokenizer.encode(sentence, max_length=512, add_special_tokens=False, truncation=True, return_tensors="pt")[0] for sentence in pred_str]
        pred_idfs = [torch.Tensor([idf_scores.get(token, 0) for token in sentence]) for sentence in pred_idfs]
        pred_idfs = torch.nn.utils.rnn.pad_sequence(pred_idfs, batch_first=True, padding_value=0)
        
        target_idfs = [tokenizer.encode(sentence, max_length=512, add_special_tokens=False, truncation=True, return_tensors="pt")[0] for sentence in target_str]
        target_idfs = [torch.Tensor([idf_scores.get(token, 0) for token in sentence]) for sentence in target_idfs]
        target_idfs = torch.nn.utils.rnn.pad_sequence(target_idfs, batch_first=True, padding_value=0)

        # pred_idfs = torch.Tensor(
        #     [[idf_scores.get(token, 0) for token in sentence] for sentence in pred_str]
        # )
        
        # target_idfs = torch.Tensor(
        #     [[idf_scores.get(token, 0) for token in sentence] for sentence in target_str]
        # )
        
        # cos_sims = pairwise_cosine_similarity(target_embedd, pred_embedd)
        
        # max_sims = torch.max(cos_sims, dim=1)[0]
        # sentence_scores.append(sum(max_sims*target_idf)/sum(target_idf))
        
        prec, res, f1 = _get_precision_recall_f1(pred_embedd, target_embedd, pred_idfs, target_idfs)
        precs.append(torch.mean(prec))
        recals.append(torch.mean(res))
        f1s.append(torch.mean(f1))
        
    return  sum(precs) / len(precs), \
            sum(recals) / len(recals), \
            sum(f1s) / len(f1s)
        

In [7]:
def update(self, preds: List[str], targets: List[str]) -> None:
        """Store predictions/references for computing BERT scores.

        It is necessary to store sentences in a tokenized form to ensure the DDP mode working.
        """
        
        preds_tokenized_data = self.tokenizer(preds, max_length=1024, truncation=True, return_tensors="pt", padding=True)
        preds_dict = {"input_ids": preds_tokenized_data["input_ids"], "attention_mask": preds_tokenized_data["attention_mask"]}
        target_tokenized_data = self.tokenizer(targets, max_length=1024, truncation=True, return_tensors="pt", padding=True)
        target_dict = {"input_ids": target_tokenized_data["input_ids"], "attention_mask": target_tokenized_data["attention_mask"]}

        self.preds_input_ids.append(preds_dict["input_ids"])
        self.preds_attention_mask.append(preds_dict["attention_mask"])
        self.target_input_ids.append(target_dict["input_ids"])
        self.target_attention_mask.append(target_dict["attention_mask"])
        
def compute_bert_score2(target_sentences : Iterable[str], pred_senteces : Iterable[str], tokenizer, model : torch.nn.Module, *args, **kwargs):
    BERTScore.update = update
    user_forward_fn = lambda model, d: model.get_input_embeddings()(d["input_ids"])
    bert_metric = BERTScore(model=model, user_tokenizer=tokenizer, user_forward_fn=user_forward_fn, device=DEVICE, verbose=True, num_layers=0, max_length=1024)
    # bert_metric.update = update
    output = {}
    for key, vals in bert_metric(preds=pred_senteces, targets=target_sentences).items():
        output[key] = float(np.mean(vals))
    
    return output
#     bert_metric.update(bert_metric, pred_senteces, target_sentences)
#     return bert_metric.compute()

In [14]:
def get_kernel_prefixes(kernel : str) -> Set[str]:
    prefixes = set()
    one_line_kernel = kernel.replace("\n", " ")
    cuda_header_prefix_re = re.compile("__(host|global|device)__")
    
    prefixes.update(cuda_header_prefix_re.findall(one_line_kernel))
    return prefixes
        

In [21]:
def compute_metrics(targets : List[str], preds : List[str], tokenizer, model : torch.nn.Module, used_ratio : float = 1.0):
    assert used_ratio > 0 and used_ratio <= 1
    assert len(targets) == len(preds)
    
    samples = list(zip(targets, preds))
    np.random.shuffle(samples)
    n = round(len(samples) * used_ratio)
        
    metrics_d = {
        "device" : {},
        "host" : {},
        "global" : {}
    }
    
    device_sentences = []
    host_sentences = []
    global_sentences = []
    
    pb = tqdm(zip(targets, preds))
    i = 0
    for target, pred in pb:
        kernel = validation_db.find_one({"body" : target})
        if kernel is None:
            kernel = train_db.find_one({"body" : target})
            if kernel is None:
                continue
        
        prefixes = get_kernel_prefixes(kernel["header"])
        if len(prefixes) == 0:
            continue
        
        if "device" in prefixes:
            device_sentences.append((target, pred))
        if "host" in prefixes:
            host_sentences.append((target, pred))
        if "global" in prefixes:
            global_sentences.append((target, pred))
        
        i += 1
        if i >= n:
            break
    
    if len(device_sentences) == 0:
        print("WARNING: no device sample")
        # metrics_d["__device__"] = {"rouge" : 0, "bleu" : 0, "bert" : 0}
    else:
        device_targets, device_preds = zip(*device_sentences)
        metrics_d["device"] = {
            "rouge" : compute_rouge_score(device_targets, device_preds, tokenizer),
            "bleu" : compute_bleu_score(device_targets, device_preds, tokenizer),
            "bert" : compute_bert_score2(device_targets, device_preds, tokenizer, model)
        }
        print(f"__device__ ({len(device_sentences)})")
        pprint(metrics_d["device"])
        
    if len(host_sentences) == 0:
        print("WARNING: no host samples")
        # metrics_d["__host__"] = {"rouge" : 0, "bleu" : 0, "bert" : 0}
    else:
        host_targets, host_preds = zip(*host_sentences)
        metrics_d["host"] = {
            "rouge" : compute_rouge_score(host_targets, host_preds, tokenizer),
            "bleu" : compute_bleu_score(host_targets, host_preds, tokenizer),
            "bert" : compute_bert_score2(host_targets, host_preds, tokenizer, model)
        }
        print(f"__host__ ({len(host_sentences)})")
        pprint(metrics_d["host"])
        
    if len(global_sentences) == 0:
        print("WARNING: no global samples")
        # metrics_d["__global__"] = {"rouge" : 0, "bleu" : 0, "bert" : 0}
    else:
        global_targets, global_preds = zip(*global_sentences)
        metrics_d["global"] = {
            "rouge" : compute_rouge_score(global_targets, global_preds, tokenizer),
            "bleu" : compute_bleu_score(global_targets, global_preds, tokenizer),
            "bert" : compute_bert_score2(global_targets, global_preds, tokenizer, model)
        }
        print(f"__global__ ({len(global_sentences)})")
        pprint(metrics_d["global"])
        
    # Weights 
    total = len(device_sentences) + len(host_sentences) + len(global_sentences)
    d_w = len(device_sentences) / total
    h_w = len(host_sentences) / total
    g_w = len(global_sentences) / total
    
    def calculate_total_metric(ds : List[dict], ws : List[float]):        
        def calculate(ds : List[dict], ws : List[float], key : str):
            r = 0
            for d, w in zip(ds, ws):
                if d is None:
                    continue
                
                r += d.get(key, 0) * w
            return r
        
        total_rouge = {}
        
        for d in ds:
            if d is None:
                continue
            for key in d.keys():
                if key in total_rouge:
                    continue
                
                total_rouge[key] = calculate(ds, ws, key)
        return total_rouge
    
    metrics_d["total"] = {"rouge" : {}, "bleu" : -1, "bert" : {}}
    metrics_d["total"]["rouge"] = calculate_total_metric(
        [metrics_d["device"].get("rouge"), metrics_d["host"].get("rouge"), metrics_d["global"].get("rouge")],
        [d_w, h_w, g_w]
    )
    
    metrics_d["total"]["bleu"] = metrics_d["device"].get("bleu",0)*d_w + metrics_d["host"].get("bleu",0)*h_w + metrics_d["global"].get("bleu",0)*g_w

    
    metrics_d["total"]["bert"] = calculate_total_metric(
        [metrics_d["device"].get("bert"), metrics_d["host"].get("bert"), metrics_d["global"].get("bert")],
        [d_w, h_w, g_w]
    )
    
    print("total")
    pprint(metrics_d["total"])
    
    # rouge = compute_rouge_score(targets, preds, tokenizer)
    # pprint({"rouge" : rouge})

    # bleu = compute_bleu_score(targets, preds, tokenizer)
    # pprint({"bleu" : bleu})

    # bert = compute_bert_score2(targets, preds, tokenizer, model)
    # pprint({"bert" : bert})
    
    return metrics_d

# CodeGen

In [3]:
def get_codegen_model(model_name : str, model_path : str, tokenizer):

    configuration = AutoConfig.from_pretrained(model_name)

    model = AutoModelForCausalLM.from_config(configuration).to(DEVICE)
    model.resize_token_embeddings(51200)
    model_dict = torch.load(model_path, map_location=DEVICE)
    # model_state_dict = OrderedDict()
    # for key, val in model_dict["model_dict"].items():
    #     model_state_dict[".".join(key.split(".")[1:])] = val
    # model.load_state_dict(model_state_dict)
    model.load_state_dict(model_dict["model_dict"])
    
    return model, model_dict

In [7]:
codegen_model_name = "Salesforce/codegen-350M-multi"
codegen_model_path = "/home/xsaman02/Project/models/codeGen/codegen.evaluated.pt"
codegen_tokenizer_name = "Salesforce/codegen-350M-multi"
tokenizer = AutoTokenizer.from_pretrained(codegen_tokenizer_name, use_fast=False, model_max_length=800, add_bos_token=True)
tokenizer.padding_side = "left"
tokenizer.model_max_length=800
tokenizer.add_special_tokens({
    "pad_token" : "</s>"
})
model, model_d = get_codegen_model(codegen_model_name, codegen_model_path, tokenizer)


In [11]:
# print(model_d.get("target_sentences")[0])
print(model_d.get("predic_sentences"))


['', '', '', '', '', '', '', '', '', '', '', 'etTextureAlignmentOffset(size_t*, const struct texture<T, dim, readMode>&) "cudaGetTextureAlignmentOffset (C++ API)"\n\ntemplate<class T, int dim, enum cudaTextureReadMode readMode>\nstatic __CUDA_DEPRECATED __inline__ __host__ cudaError_t cudaBindTexture(\n        size_t                           *offset,\n  const struct texture<T, dim, readMode> &tex,\n  const void                             *devPtr,\n        size_t                            size = UINT_MAX\n)', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [8]:
used_ratio = 0.1
metrics = compute_metrics(model_d.get("target_sentences"), model_d.get("pred_sentences"), tokenizer, model, used_ratio)

NameError: name 'compute_metrics' is not defined

Epochs: 20
Max: 2.3, epoch: 0
Min: 0.12702270974046703, epoch: 19


: 

In [16]:
loss_list = model_d.get("loss_list")
loss_list[0] = 2.3
plt.figure()
plt.title("CodeGen loss")
plt.ylabel("Cross-entropy loss")
plt.xlabel("Epoch")
plt.plot(loss_list)
plt.show()

print(f"Epochs: {len(loss_list)}")
print(f"Max: {max(loss_list)}, epoch: {np.argmax(loss_list)}")
print(f"Min: {min(loss_list)}, epoch: {np.argmin(loss_list)}")

### Average lengths

In [5]:
tgr_ls = 0
tgr_cs = 0
prd_ls = 0
prd_cs = 0
s = len(model_d.get("source_sentences"))
for tgr, prd in tqdm(zip(model_d.get("source_sentences"), model_d.get("target_sentences"))):
    tgr = tgr.replace(";", ";\n").replace("{", "{\n").replace("}", "}\n").replace("int", "\nint").replace("float", "\nfloat")
    prd = prd.replace(";", ";\n").replace("{", "{\n").replace("}", "}\n").replace("int", "\nint").replace("float", "\nfloat")
    
    tgr_ls += tgr.count("\n")
    prd_ls += prd.count("\n")
    
    tgr_cs += len(tgr)
    prd_cs += len(prd)
    
print("Average target lines per sample: {:.3f}".format(tgr_ls/s))
print("Average predic lines per sample: {:.3f}".format(prd_ls/s))
print("Average target sample size: {:.3f}".format(tgr_cs/s))
print("Average predic sample size: {:.3f}".format(prd_cs/s))


0it [00:00, ?it/s]

Average target lines per sample: 7.790
Average predic lines per sample: 39.776
Average target sample size: 326.022
Average predic sample size: 685.084


### Examples

In [30]:
# Generate random example from evaluation
samples = list(zip(model_d.get("source_sentences"), model_d.get("target_sentences"), model_d.get("pred_sentences")))
np.random.shuffle(samples)
src_l, tgr_l, prd_l = zip(*samples)
for src, tgr, prd in zip(src_l, tgr_l, prd_l):
    kernel = train_db.find_one({"body" : tgr})
    if kernel is None:
        kernel = validation_db.find_one({"body" : tgr})
        if kernel is None:
            continue
    
    print("SOURCE:")
    print(src)
    print("\nTARGET:")
    print(tgr)
    print("\nPREDICT:")
    print(prd.replace(";", ";\n").replace("{", "{\n").replace("}", "}\n").replace("int", "\nint").replace("float", "\nfloat"))
    print("#--------------------------#")
    break

SOURCE:
supplement code:// kernels
template <typename T>
__global__ void clMatLinspace_kernel(IN OUT T* pA, IN T a, IN T d, int dim)

TARGET:
{
  const unsigned int ind = blockIdx.x*blockDim.x + threadIdx.x;
  const unsigned int totalThreads = blockDim.x*gridDim.x;
  for (int i = ind; i < dim; i += totalThreads)
    pA[i] = a+i*d;
}

PREDICT:
{
 
int i = threadIdx.x + blockIdx.x * blockDim.x;
 if (i < dim) {
 pA[i] = a[i] + dim;
 }
 }

#--------------------------#
